<a href="https://colab.research.google.com/github/lihiShlomit/DSproject/blob/main/license_plates_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Connecting to google drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Setting directories

import os
import re

project_dir = "/content/drive/MyDrive/project"

videos_dir = os.path.join(project_dir, "videos")

frames_dir = os.path.join(project_dir, "frames")

with_boxes_dir = os.path.join(project_dir, "with_boxes")

os.makedirs(videos_dir, exist_ok=True)

os.makedirs(frames_dir, exist_ok=True)

os.makedirs(with_boxes_dir, exist_ok=True)

In [ ]:
# @title Creating a directory for the new video

existing_videos = []
for name in os.listdir(videos_dir):
    match = re.match(r"video(\d+)$", name)
    if match and os.path.isdir(os.path.join(videos_dir, name)):
        existing_videos.append(int(match.group(1)))

if existing_videos:
    next_number = max(existing_videos) + 1
else:
    next_number = 1

new_dir_name = f"video{next_number}"
video_dir = os.path.join(videos_dir, new_dir_name)
os.makedirs(video_dir, exist_ok=True)

In [ ]:
# @title Needed imports for uploading a video

import os
import shutil
from google.colab import files
from moviepy.editor import VideoFileClip
from PIL import Image

video_filename = "video.mp4"
video_path = os.path.join(video_dir, "video.mp4")

In [ ]:
# @title Upload a video

uploaded = files.upload()
for filename in uploaded.keys():
    shutil.move(filename, video_path)
print(f"Video uploaded to: {video_path}")

In [ ]:
# @title Needed imports and settings to extract frames from videos

from moviepy.editor import VideoFileClip
from PIL import Image

clip = VideoFileClip(video_path)
frame_rate = clip.fps
duration = clip.duration
frame_count = int(duration * frame_rate)
frames_dir = os.path.join(video_dir, "frames")
os.makedirs(frames_dir, exist_ok=True)

In [ ]:
# @title Extract frames from video

for i in range(frame_count):
    frame_time = i / frame_rate
    frame = clip.get_frame(frame_time)
    frame_path = os.path.join(frames_dir, f"frame_{i:05d}.jpg")
    Image.fromarray(frame).save(frame_path)

print(f"finished extraction")

In [ ]:
# @title needed installations

!pip install -q ultralytics opencv-python

In [ ]:
# @title Cloning and installing yolov5

!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

In [ ]:
# @title imports for marking a bounding box

import torch
import os
import cv2
from pathlib import Path

In [ ]:
# @title Function that creates bounding boxes around the license plates

def plot_one_box(x, img, color=(128, 128, 128), label=None, line_thickness=4):
    tl = line_thickness or int(round(0.002 * max(img.shape[0:2])))
    x1, y1, x2, y2 = map(int, x)
    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)
        font_scale = 0.5
        t_size = cv2.getTextSize(label, 0, fontScale=font_scale, thickness=tf)[0]
        c2 = x1 + t_size[0], y1 - t_size[1] - 3
        cv2.rectangle(img, (x1, y1), c2, color, -1, cv2.LINE_AA)
        cv2.putText(img, label, (x1, y1 - 2), 0, font_scale, [255, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

In [ ]:
# @title Imports and settings for Identifying cars with yolov5

frames_with_boxes_dir = os.path.join(video_dir, "with_boxes")
os.makedirs(frames_with_boxes_dir, exist_ok=True)

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

cars_crops_dir = os.path.join(video_dir, "cars_crops")
os.makedirs(cars_crops_dir, exist_ok=True)

In [ ]:
# @title Identifying cars and creating bounding boxes around them

frames_with_boxes_dir = os.path.join(video_dir, "with_boxes")
os.makedirs(frames_with_boxes_dir, exist_ok=True)

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

cars_crops_dir = os.path.join(video_dir, "cars_crops")
os.makedirs(cars_crops_dir, exist_ok=True)

count=0
for filename in sorted(os.listdir(frames_dir)):
    if filename.endswith(".jpg"):
        img_path = os.path.join(frames_dir, filename)
        img = cv2.imread(img_path)

        results = model(img)

        car_count = 0
        for *box, conf, cls in results.xyxy[0].tolist():
            cls = int(cls)
            label = results.names[cls]
            if label == "car":
                x1, y1, x2, y2 = map(int, box)
                car_crop = img[y1:y2, x1:x2]
                crop_filename = f"{os.path.splitext(filename)[0]}_car{car_count}.jpg"
                crop_path = os.path.join(cars_crops_dir, crop_filename)
                cv2.imwrite(crop_path, car_crop)

                car_count += 1

        for *box, conf, cls in results.xyxy[0].tolist():
            cls = int(cls)
            label = results.names[cls]
            if label == "car":
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        output_path = os.path.join(frames_with_boxes_dir, filename)
        cv2.imwrite(output_path, img)

print(f"all cars marked")

In [ ]:
# @title Loading the fine-tuned model, that identifies license plates

lp_model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/project/license_plates_model.pt')

In [ ]:
# @title Installations to allow use of OCR

!apt-get install -y tesseract-ocr
!pip install pytesseract

In [ ]:
# @title Imports for placing bound boxes

import os
import cv2
import pytesseract
import re

license_plates_with_boxes_dir = os.path.join(video_dir, "license_plates_with_boxes")
os.makedirs(license_plates_with_boxes_dir, exist_ok=True)

video3_frames = os.path.join(video_dir, "frames")

In [ ]:
# @title Identifing license plates, place bounding boxes around them and around the cars in the origional pictures, and run OCR on them

car_count = 0

for filename in sorted(os.listdir(video3_frames)):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(video3_frames, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue

        img_h, img_w = img.shape[:2]
        results = model(img)
        car_count = 0

        for *box, conf, cls in results.xyxy[0].tolist():
            cls = int(cls)
            label = results.names[cls]
            if label == "car":
                x1, y1, x2, y2 = map(int, box)

                # car boundries
                x1 = max(0, min(img_w, x1))
                x2 = max(0, min(img_w, x2))
                y1 = max(0, min(img_h, y1))
                y2 = max(0, min(img_h, y2))

                if x2 <= x1 or y2 <= y1:
                    continue

                car_crop = img[y1:y2, x1:x2]
                if car_crop.size == 0:
                    continue

                lp_results = lp_model(car_crop)
                ch, cw = car_crop.shape[:2]

                for *lp_box, lp_conf, lp_cls in lp_results.xyxy[0].tolist():
                    lx1, ly1, lx2, ly2 = map(int, lp_box)

                    # plate boundries
                    lx1 = max(0, min(cw, lx1))
                    lx2 = max(0, min(cw, lx2))
                    ly1 = max(0, min(ch, ly1))
                    ly2 = max(0, min(ch, ly2))

                    if lx2 <= lx1 or ly2 <= ly1:
                        continue

                    license_plate_crop = car_crop[ly1:ly2, lx1:lx2]
                    if license_plate_crop.size == 0:
                        continue

                    raw_text = pytesseract.image_to_string(license_plate_crop, config='--psm 7').strip()
                    lp_text = re.sub(r'[^0-9]', '', raw_text).strip()

                    #prevent the ocr from identifying a part of the plate as the number on it
                    if len(lp_text) > 8 and lp_text.startswith("1"):
                        lp_text = lp_text[1:]

                    abs_lx1 = x1 + lx1
                    abs_ly1 = y1 + ly1
                    abs_lx2 = x1 + lx2
                    abs_ly2 = y1 + ly2

                    cv2.rectangle(img, (abs_lx1, abs_ly1), (abs_lx2, abs_ly2), (0, 255, 255), 2)
                    cv2.putText(img, lp_text, (abs_lx1, abs_ly1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                                0.8, (0, 255, 255), 2, lineType=cv2.LINE_AA)

                plot_one_box([x1, y1, x2, y2], img, label=label, color=(0, 255, 0), line_thickness=4)
                car_count += 1

        output_path = os.path.join(license_plates_with_boxes_dir, filename)
        cv2.imwrite(output_path, img)

print("All frames processed and saved with license plates and texts.")


In [ ]:
# @title Combine frames into a video

from moviepy.editor import VideoFileClip
video_path = os.path.join(video_dir, video_filename)
clip = VideoFileClip(video_path)

from moviepy.editor import ImageSequenceClip
import os

frames_with_boxes_dir = license_plates_with_boxes_dir

frame_files = sorted([os.path.join(frames_with_boxes_dir, f)
                      for f in os.listdir(frames_with_boxes_dir)
                      if f.endswith(".jpg") or f.endswith(".png")])

fps = clip.fps
video_clip = ImageSequenceClip(frame_files, fps=fps)

output_video_path = os.path.join(video_dir, "output_with_boxes.mp4")
video_clip.write_videofile(output_video_path, codec="libx264")


In [ ]:
# @title Presenting the video

from IPython.display import Video, display

display(Video(output_video_path, embed=True, width=640))